In [ ]:
"""
RAMS query program

Created on Wed Jul 5 11:49:17 2017

@author: Tingting Huang

thuang1@iastate.edu

"""

In [1]:
# use your own directory and filename

# Must use column name: Latitude, Longitude, routeId (in LRS format, e.g. "S001910235E" for I-235 EB)


dirpath = 'C:/Users/thuang1/Desktop/'
filename = 'RouteMM_SensorList2.csv'



In [2]:
import pandas as pd
import numpy as np
import json
import requests
from datetime import datetime

In [3]:
def get_locations(df):
    d = []
    for i in xrange(len(df)):
        x = {'geometry':{'y':df.iloc[i]['Latitude'], 'x':df.iloc[i]['Longitude']}, 'routeId':df.iloc[i]['routeId']}
        d.append(x)

    locations = json.dumps(d, ensure_ascii=False)

    return locations

In [29]:
def get_measure(locations, tolerance):
       

    url = 'https://gis.iowadot.gov/ramsa/rest/services/lrs/MapServer/exts/LRSServer/networkLayers/0/geometryToMeasure?'
    data = {'f':'json','locations': locations,'tolerance':str(tolerance),'inSR':'4326'}
    response = requests.post(url, data=data)
    r=response.json()

    # extract results
    dat=[]
    for i in range(len(r['locations'])):
        if r['locations'][i]['status']!=u'esriLocatingOK':
            print "Error at Line:", i
            dat.append(' ')
        else:
            dat.append(r['locations'][i]['results'][0]['measure'])
    dat=np.asarray(dat)
    
    return dat

In [5]:
# load your data
df = pd.read_csv(dirpath+filename)

n=500

ans=[]
for i in xrange(len(df)/n+1):
    ans.append(df[i*n:i*n+n])

In [30]:
# Main
now1=datetime.now()

final=[]
i=0

for a in ans:
    i=i+1
    locations = get_locations(a)
    
    dat = get_measure(locations, 50)
        
    a['MM']=dat
    
    final.append(a)
    
    print('Progress: '+str(round(float(i)/len(ans)*100,2))+"%")

final=pd.concat(final,axis=0).reset_index(drop=True)

now2=datetime.now()

print('Runtime: '+str(now2-now1))

Error at Line: 145
Error at Line: 146
Success! Progress: 100.0%
Runtime: 0:00:04.310000


C:\Users\thuang1\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [31]:
# save to new file
final.to_csv(dirpath+'MM_Found_'+filename, header=True, index=False)